In [ ]:
import numpy as np
import sys
import os
from array import array

import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score
 
from struct import *
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
train_lbl = [ [],[],[],[],[],[],[],[],[],[] ] #숫자별로 저장 (0 ~ 9)
test_lbl = [ [],[],[],[],[],[],[],[],[],[] ] #숫자별로 저장 (0 ~ 9)

In [ ]:
def read_dataset(path_image, path_label, data_type): #파일 읽기 
    mnist_image_origin = open(path_image,'rb')
    mnist_label_origin = open(path_label,'rb')

    #사용할 변수 초기화
    mnist_img = np.zeros((28,28)) #이미지가 저장될 부분

    label = 0
    index=0 
    count_row = 0

    first_image_byte = mnist_image_origin.read(16)    #read first 16byte
    first_label_byte = mnist_label_origin.read(8)     #read first  8byte

    test_img = None

    while True:    
        image = mnist_image_origin.read(784) #784바이트(= 28*28) 씩 읽음
        label = mnist_label_origin.read(1)   #1바이트씩 읽음

        if not image:
            break; 
        if not label:
            break;

        index = int(label[0])      

        #unpack
        mnist_img = np.reshape( unpack(len(image)*'B',image), (28,28)) 
        
        if data_type == 'train':
            train_lbl[index].append(mnist_img) #각 숫자영역별로 해당이미지를 추가
        else:
            test_lbl[index].append(mnist_img) #각 숫자영역별로 해당이미지를 추가
        count_row = count_row + 1

    print("read done " + str(count_row))
    
    return train_lbl if data_type == 'train' else test_lbl

In [ ]:
def get_row(label, index_num, data_type):
    
    change_count = 0
    pre_val = 0
    feature = []
    featured_image = pd.DataFrame(columns = ["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","x13","x14","x15",
                                             "x16","x17","x18","x19","x20","x21","x22","x23","x24","x25","x26","x27","label"])
    
    check_img = None
    
    if data_type == 'train':
            check_img = train_lbl[int(label)][int(index_num)]
    else:
        check_img = test_lbl[int(label)][int(index_num)]
    

    for r in range(0,27):
        change_count = 0
        pre_val = 0

        for c in range(1,27):
            if check_img[r][c] == 0 and check_img[r][pre_val] > 0:
                change_count = change_count + 1
            elif check_img[r][c] > 0 and check_img[r][pre_val] == 0:
                change_count = change_count + 1

            pre_val = pre_val + 1

        feature.append(change_count)    

    feature.append(label)
    featured_image.loc[0] = pd.Series(feature,index=featured_image.columns)
    
    return featured_image

In [ ]:
train_mnist = read_dataset('mnist/train-images.idx3-ubyte','mnist/train-labels.idx1-ubyte', 'train')
test_mnist = read_dataset('mnist/t10k-images.idx3-ubyte','mnist/t10k-labels.idx1-ubyte', 'test')

In [ ]:
def prep_work(number, return_df):
        
    # 전체 mnist 전처리 
    mnist_train_data = pd.DataFrame(columns = ["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","x13","x14","x15",
                                               "x16","x17","x18","x19","x20","x21","x22","x23","x24","x25","x26","x27","label"])

    for n in range(0,len(train_mnist[number])):
        mnist_train_data = pd.concat([mnist_train_data, get_row(i,n,'train')], ignore_index=True)
   
    return_df[number] = mnist_train_data

In [ ]:
import multiprocessing
manager = multiprocessing.Manager()
return_df = manager.dict()

procs = []
for i in range(10):
    p = multiprocessing.Process(target=prep_work, args=(i, return_df))
    p.start()
    procs.append(p)

for p in procs:
    p.join()  # 프로세스가 모두 종료될 때까지 대기

In [ ]:
mnist_train_data = pd.DataFrame(columns = ["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","x13","x14","x15",
                                           "x16","x17","x18","x19","x20","x21","x22","x23","x24","x25","x26","x27","label"])

for i in range(10):
    mnist_train_data = mnist_train_data = pd.concat([mnist_train_data, return_df[i]], ignore_index=True)

mnist_train_data.shape

In [ ]:
# 전체 mnist 전처리 
mnist_test_data = pd.DataFrame(columns = ["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","x13","x14","x15",
                                           "x16","x17","x18","x19","x20","x21","x22","x23","x24","x25","x26","x27","label"])

for i in range(0,10):    
    for n in range(0,len(test_mnist[i])):
        mnist_test_data = pd.concat([mnist_test_data, get_row(i,n,'test')], ignore_index=True)

mnist_test_data.shape

In [ ]:
plt.imshow(train_lbl[7][0], cmap = cm.binary) #binary형태의 이미지 설정
plt.show()
print(test_lbl[7][0])

In [ ]:
X = mnist_train_data[mnist_train_data.columns[:-1]]
y = mnist_train_data['label']
y = y.astype('int')

# SVM 모델 생성
model = SVC(kernel='poly', C = 3, degree = 3)
model.fit(X, y)

In [ ]:
test_X = mnist_test_data[mnist_test_data.columns[:-1]]
test_y = mnist_test_data['label']
test_y = test_y.astype('int')

y_pred = model.predict(test_X)

f1_result = f1_score(test_y, y_pred, average = None)

print(f1_result)
print(np.mean(f1_result))

In [ ]:
confusion_matrix(test_y, y_pred)